In [ ]:
import os
import xml.etree.ElementTree as ET

def parse_xml(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    # Extract the main text
    main_text = ' '.join([elem.text for elem in root.findall('.//section') if elem.text])
    return main_text

# Example usage
file_path = 'ScisummNet/data/SomePaper/SomePaper.xml'
main_text = parse_xml(file_path)
print(main_text[:500])  # Print the first 500 characters


In [1]:
from transformers import AutoTokenizer

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-small")

# Function to preprocess the data
def preprocess_data(text, summary):
    model_inputs = tokenizer(text, max_length=1024, truncation=True, padding="max_length")
    labels = tokenizer(summary, max_length=150, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Example usage
text = "Your extracted main text here."
summary = "Corresponding summary here."
processed_data = preprocess_data(text, summary)


c:\Users\ASUS\xStyNWx\Documents\BINUS University\Academic Courses\Semester 4\Natural Language Processing\Project\Repo\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import T5ForConditionalGeneration

# Load the pre-trained T5 model
model = T5ForConditionalGeneration.from_pretrained("t5-small")


In [3]:
from transformers import Seq2SeqTrainingArguments

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True
)


c:\Users\ASUS\xStyNWx\Documents\BINUS University\Academic Courses\Semester 4\Natural Language Processing\Project\Repo\venv\Lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [4]:
from torch.utils.data import Dataset

class SummarizationDataset(Dataset):
    def __init__(self, texts, summaries, tokenizer, max_input_length=1024, max_output_length=150):
        self.texts = texts
        self.summaries = summaries
        self.tokenizer = tokenizer
        self.max_input_length = max_input_length
        self.max_output_length = max_output_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        summary = self.summaries[idx]
        model_inputs = self.tokenizer(text, max_length=self.max_input_length, truncation=True, padding="max_length", return_tensors="pt")
        labels = self.tokenizer(summary, max_length=self.max_output_length, truncation=True, padding="max_length", return_tensors="pt")
        model_inputs["labels"] = labels["input_ids"].squeeze()
        return {key: val.squeeze() for key, val in model_inputs.items()}

# Example usage
texts = ["Your extracted main text here."]
summaries = ["Corresponding summary here."]
dataset = SummarizationDataset(texts, summaries, tokenizer)


In [5]:
from transformers import Seq2SeqTrainer, DataCollatorForSeq2Seq

# Initialize data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Initialize the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=dataset,  # Ideally, use a separate validation set
    tokenizer=tokenizer,
    data_collator=data_collator
)


C:\Users\ASUS\AppData\Local\Temp\ipykernel_22012\3078878853.py:7: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [6]:
# Train the model
trainer.train()


c:\Users\ASUS\xStyNWx\Documents\BINUS University\Academic Courses\Semester 4\Natural Language Processing\Project\Repo\venv\Lib\site-packages\transformers\data\data_collator.py:740: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:257.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,No log,11.092548
2,No log,10.770166
3,No log,10.604973


TrainOutput(global_step=3, training_loss=9.943681081136068, metrics={'train_runtime': 10.0909, 'train_samples_per_second': 0.297, 'train_steps_per_second': 0.297, 'total_flos': 812050808832.0, 'train_loss': 9.943681081136068, 'epoch': 3.0})

In [7]:
# Evaluate the model
results = trainer.evaluate()

# Print evaluation results
print(results)


{'eval_loss': 10.604972839355469, 'eval_runtime': 0.4934, 'eval_samples_per_second': 2.027, 'eval_steps_per_second': 2.027, 'epoch': 3.0}


In [8]:
# Function to generate summary
def generate_summary(text):
    inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs.input_ids, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)

::contentReference[oaicite:35]{index=35}
 


SyntaxError: invalid syntax (2060044223.py, line 6)